In [ ]:
from sae_lens import SAE, ActivationsStore
from transformer_lens import HookedTransformer

In [ ]:
def load_model_and_sae(
    model_name: str, sae_release: str, sae_id: str, device: str
) -> tuple:
    model = HookedTransformer.from_pretrained(model_name, device=device)
    sae, _, _ = SAE.from_pretrained(release=sae_release, sae_id=sae_id, device=device)
    sae.W_dec.norm(dim=-1).mean()
    sae.fold_W_dec_norm()
    return model, sae

In [ ]:
model_gemma, sae_gemma = load_model_and_sae(
    "gemma-2-2b",
    "gemma-scope-2b-pt-res-canonical",
    "layer_18/width_16k/canonical",
    "mps",
)

In [ ]:
model_gpt2, sae_gpt2 = load_model_and_sae(
    "gpt2-small",
    "gpt2-small-res-jb-feature-splitting",
    "blocks.8.hook_resid_pre_768",
    "mps",
)

In [ ]:
activation_store_gpt2 = ActivationsStore.from_sae(
    model=model_gpt2,
    sae=sae_gpt2,
    streaming=True,
    store_batch_size_prompts=8,
    train_batch_size_tokens=512,
    n_batches_in_buffer=8,
    device="mps",
)

In [ ]:
activation_store_gemma = ActivationsStore.from_sae(
    model=model_gemma,
    sae=sae_gemma,
    streaming=True,
    store_batch_size_prompts=8,
    train_batch_size_tokens=512,
    n_batches_in_buffer=8,
    device="mps",
)